<a href="https://colab.research.google.com/github/kingsman0909/MyPortfolio/blob/main/bark_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 🧼 Clean up Colab environment to avoid conflicts
!pip uninstall -y tensorflow jax jaxlib xarray arviz pymc opencv-python opencv-python-headless thinc treescope sentence-transformers

# 🔁 Reinstall compatible NumPy version
!pip install numpy==1.24.4 --force-reinstall

# 🔄 Install compatible PyTorch version (CUDA 11.8)
!pip install torch==2.0.1+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

# 📦 Clone Tortoise TTS
!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts

# 🧪 Install required packages
!pip install -r requirements.txt
!pip install einops scipy tqdm

# 📥 Download the model weights
!python tortoise/utils/download_models.py


# ✅ Now import and run Tortoise TTS
import torchaudio
import torch
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_voices  # For prebuilt voice presets

# 🗣️ Choose a voice (pat is stable)
voice_type = "pat"  # Options: 'pat', 'daniel', 'freeman', 'emma', 'random'

# ✍️ Text to generate
text = "My body may fall—but my will won't."

# 🔥 Load TTS engine and voice preset
tts = TextToSpeech()
voice_samples, conditioning_latents = load_voices([voice_type])

# 🎧 Generate audio using ultra-fast preset
output = tts.tts_with_preset(
    text,
    voice_samples=voice_samples,
    conditioning_latents=conditioning_latents,
    preset='ultra_fast'  # Try 'high_quality' for better audio
)

# 💾 Save result
torchaudio.save('thorfin_output.wav', output.squeeze(0).cpu(), 24000)
print("✅ Saved as thorfin_output.wav")


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 73.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.7.1 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.24.4 which is incompatible.
arviz 0.22.0 requires numpy>=1.26.0, but you have numpy 1.24.4 which is incompatible.
scipy 1.16.0 requires numpy<2.6,>=1.25.2, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 2001, done.
remote: Total 2001 (delta 0), reused 0 (delta 0), pack-reused 2001 (from 1)
Receiving objects: 100% (2001/2001), 54.20 MiB | 10.24 MiB/s, done.
Resolving deltas: 100% (916/916), done.
/content/tortoise-tts/tortoise-tts
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 98.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.2 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_ver

python3: can't open file '/content/tortoise-tts/tortoise-tts/tortoise/utils/download_models.py': [Errno 2] No such file or directory


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'unidecode'